# Machine Learning in Spark

In [5]:
import pyspark

# un-comment the following lines if running locally
spark = pyspark.sql.SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [2]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml import feature
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoderEstimator

In [3]:
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline

This example assumes that we have a holdout validation dataset somewhere else, so we don't need to perform a train-test split, we only need to perform cross validation

Follow [these instructions](https://docs.databricks.com/data/data.html#import-data-1) to import `US_births_2000-2014_SSA.csv` into Databricks

In [6]:
# this file path will be different if you are running Spark locally
df = spark.read.format('csv').option('header', 'true').\
load('/FileStore/tables/US_births_2000_2014_SSA-daa0e.csv')

AnalysisException: 'Path does not exist: file:/FileStore/tables/US_births_2000_2014_SSA-daa0e.csv;'

In [7]:
df.toPandas().head(3)

,year,month,date_of_month,day_of_week,births
0,2000,1,1,6,9083
1,2000,1,2,7,8006
2,2000,1,3,1,11363


In [8]:
df.dtypes

Out[6]: [('year', 'string'),
 ('month', 'string'),
 ('date_of_month', 'string'),
 ('day_of_week', 'string'),
 ('births', 'string')]

In [9]:
df = df.withColumn('births', df['births'].cast('int'))
df = df.withColumn('day_of_week', df['day_of_week'].cast('int'))
df = df.withColumn('date_of_month', df['date_of_month'].cast('int'))
df = df.withColumn('month', df['month'].cast('int'))
df = df.withColumn('year', df['year'].cast('int'))

In [10]:
ohe = feature.OneHotEncoderEstimator(inputCols=['date_of_month',
                                                'day_of_week'],
                                     outputCols=['date_vec',
                                                  'day_vec'],
                                     dropLast=True)
one_hot_encoded = ohe.fit(df).transform(df)
one_hot_encoded.head()

Out[8]: Row(year=2000, month=1, date_of_month=1, day_of_week=6, births=9083, date_vec=SparseVector(31, {1: 1.0}), day_vec=SparseVector(7, {6: 1.0}))

Note the 'SparseVector' we've created!

In [12]:
features = ['year', 'month', 'date_of_month', 'day_of_week']

target = 'births'

vector = VectorAssembler(inputCols=features, outputCol='features')
vectorized_df = vector.transform(one_hot_encoded)

The Vector Assembler is often what we want when we're building a model in Spark. [How does the VectorAssembler work?](https://spark.apache.org/docs/2.1.0/ml-features.html#vectorassembler)

In [14]:
vectorized_df.columns

Out[10]: ['year',
 'month',
 'date_of_month',
 'day_of_week',
 'births',
 'date_vec',
 'day_vec',
 'features']

In [15]:
rf_model = RandomForestRegressor(featuresCol='features',
                                 labelCol='births',
                                 predictionCol="prediction").fit(vectorized_df)

In [16]:
predictions = rf_model.transform(vectorized_df).select("births", "prediction")
predictions.head(3)

Out[12]: [Row(births=9083, prediction=8560.460510012514),
 Row(births=8006, prediction=7935.353253494379),
 Row(births=11363, prediction=11572.878523806241)]

Let's evaluate our model! [Here](https://spark.apache.org/docs/2.2.0/mllib-evaluation-metrics.html) is a reference for the many metrics available in Spark.

In [18]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='births')

evaluator.evaluate(predictions, {evaluator.metricName:"r2"})

Out[13]: 0.8932985136656557

In [19]:
evaluator.evaluate(predictions, {evaluator.metricName:"mae"})

Out[14]: 440.3706113891965

In [20]:
one_hot_encoder = OneHotEncoderEstimator(inputCols=['date_of_month',
                                                'day_of_week'],
                                     outputCols=['date_vec',
                                                  'day_vec'],
                                     dropLast=True)
vector_assember = VectorAssembler(inputCols=features,
                                  outputCol='features')
random_forest = RandomForestRegressor(featuresCol='features',
                                      labelCol='births')
stages = [one_hot_encoder, vector_assember, random_forest]

pipeline = Pipeline(stages=stages)

Note: The stages in a pipeline can be either *Transformers* or *Estimators*. An estimator fits a DataFrame to produce a Transformer.

In [22]:
random_forest.params

Out[16]: [Param(parent='RandomForestRegressor_918405b10230', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval.'),
 Param(parent='RandomForestRegressor_918405b10230', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext.'),
 Param(parent='RandomForestRegressor_918405b10230', name='featureSubsetStrategy', doc="The number of features to consider for splits at each tree node. Supported options: 'auto' (choose automatically for task: If numTrees == 1, set to 'all'. If numTrees > 1 (forest), set to 'sqrt' for classification and to 'onethird' for regression), 'all' (use all features), 'onethird' (use 1/3 of the features), 'sqrt' (use sqrt(number of features)), 'log2' (use log2(number of features)), 'n' (when n is in the range (0, 1.0], use n * number of features. When n is in the range (1, number of features), use n features). default = 'auto'"),
 Param(parent='RandomForestRegressor_918405b10230', name='featuresCol', doc='features column name.'),
 Param(parent='RandomForestRegressor_918405b10230', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: variance'),
 Param(parent='RandomForestRegressor_918405b10230', name='labelCol', doc='label column name.'),
 Param(parent='RandomForestRegressor_918405b10230', name='maxBins', doc='Max number of bins for discretizing continuous features. Must be >=2 and >= number of categories for any categorical feature.'),
 Param(parent='RandomForestRegressor_918405b10230', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'),
 Param(parent='RandomForestRegressor_918405b10230', name='maxMemoryInMB', doc='Maximum memory in MB allocated to histogram aggregation. If too small, then 1 node will be split per iteration, and its aggregates may exceed this size.'),
 Param(parent='RandomForestRegressor_918405b10230', name='minInfoGain', doc='Minimum information gain for a split to be considered at a tree node.'),
 Param(parent='RandomForestRegressor_918405b10230', name='minInstancesPerNode', doc='Minimum number of instances each child must have after split. If a split causes the left or right child to have fewer than minInstancesPerNode, the split will be discarded as invalid. Should be >= 1.'),
 Param(parent='RandomForestRegressor_918405b10230', name='numTrees', doc='Number of trees to train (>= 1).'),
 Param(parent='RandomForestRegressor_918405b10230', name='predictionCol', doc='prediction column name.'),
 Param(parent='RandomForestRegressor_918405b10230', name='seed', doc='random seed.'),
 Param(parent='RandomForestRegressor_918405b10230', name='subsamplingRate', doc='Fraction of the training data used for learning each decision tree, in range (0, 1].')]

In [23]:
params = ParamGridBuilder().addGrid(random_forest.maxDepth,
                                    [5,10,15]).addGrid(random_forest.numTrees,
                                                       [20,50,100]).build()

In [24]:
reg_evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='births',
                                    metricName = 'mae')

In [25]:
cv = CrossValidator(
    estimator=pipeline,
    estimatorParamMaps=params,
    evaluator=reg_evaluator,
    parallelism=4
)

In [26]:
df.show(n=5)

+----+-----+-------------+-----------+------+
year|month|date_of_month|day_of_week|births|
+----+-----+-------------+-----------+------+
2000| 1| 1| 6| 9083|
2000| 1| 2| 7| 8006|
2000| 1| 3| 1| 11363|
2000| 1| 4| 2| 13032|
2000| 1| 5| 3| 12558|
+----+-----+-------------+-----------+------+
only showing top 5 rows

In [27]:
df.limit(1000)

Out[21]: DataFrame[year: int, month: int, date_of_month: int, day_of_week: int, births: int]

In [28]:
cross_validated_model = cv.fit(df.limit(1000).cache())

/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

In [29]:
cross_validated_model.avgMetrics

Out[23]: [397.3654159596964,
 382.78840647693164,
 364.7880423340302,
 326.50061676651944,
 320.95444945771663,
 312.4293339453081,
 325.72810548924684,
 318.2149830961457,
 311.2682230162612]

In [30]:
cross_validated_model.

File "<command-3907748708933089>" , line 1 
 cross_validated_model. 
 ^
 SyntaxError : invalid syntax

## .bestModel

In [32]:
cross_validated_model.bestModel.stages

Out[25]: [OneHotEncoderEstimator_08eccbebf8d6,
 VectorAssembler_1fa3e991c82f,
 RandomForestRegressionModel (uid=RandomForestRegressor_918405b10230) with 100 trees]

In [33]:
cross_validated_model.bestModel.stages[2].getNumTrees

Out[26]: 100

## Challenge

Look at [this documentation](https://docs.databricks.com/data/databricks-datasets.html) to find large datasets that come pre-loaded on DBFS (Databricks file system).  Choose one, and build an ML model based on it.

In [35]:
display(dbutils.fs.ls("/databricks-datasets"))

path,name,size
dbfs:/databricks-datasets/,databricks-datasets/,0
dbfs:/databricks-datasets/README.md,README.md,976
dbfs:/databricks-datasets/Rdatasets/,Rdatasets/,0
dbfs:/databricks-datasets/SPARK_README.md,SPARK_README.md,3359
dbfs:/databricks-datasets/adult/,adult/,0
dbfs:/databricks-datasets/airlines/,airlines/,0
dbfs:/databricks-datasets/amazon/,amazon/,0
dbfs:/databricks-datasets/asa/,asa/,0
dbfs:/databricks-datasets/atlas_higgs/,atlas_higgs/,0
dbfs:/databricks-datasets/bikeSharing/,bikeSharing/,0


In [36]:
display(dbutils.fs.ls("/databricks-datasets/Rdatasets/"))

path,name,size
dbfs:/databricks-datasets/Rdatasets/LICENSE.txt,LICENSE.txt,58
dbfs:/databricks-datasets/Rdatasets/README.md,README.md,767
dbfs:/databricks-datasets/Rdatasets/data-001/,data-001/,0


In [37]:
with open("/databricks-datasets/Rdatasets/README.md") as f:
    x = ''.join(f.readlines())

print(x)

--------------------------------------------------------------------------- 
 FileNotFoundError Traceback (most recent call last)
 <command-151580666638183> in <module> 
 ----> 1 with open ( "/databricks-datasets/Rdatasets/README.md" ) as f : 
 2 x = '' . join ( f . readlines ( ) ) 
 3 
 4 print ( x ) 

 FileNotFoundError : [Errno 2] No such file or directory: '/databricks-datasets/Rdatasets/README.md'

^ I'm not sure why that breaks but I'm going to leave it there for now